In [15]:
from llmcoder import LLMCoder

In [16]:
llmcoder = LLMCoder(analyzers=["mypy_analyzer_v1", "signature_analyzer_v1"], max_iter=3, feedback_variant="coworker")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/CodeBERT-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
code = '''import json
import os
from datetime import datetime

import openai
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from llmcoder.analyze.factory import AnalyzerFactory
from llmcoder.utils import get_conversations_dir, get_openai_key, get_system_prompt, get_system_prompt_dir


class LLMCoder:
    def __init__(self,
                 analyzers: list[str] = None,
                 model_first: str = "ft:gpt-3.5-turbo-1106:personal::8LCi9Q0d",
                 model_feedback: str = "gpt-3.5-turbo",
                 feedback_variant: str = "separate",
                 system_prompt: str | None = None,
                 max_iter: int = 10,
                 log_conversation: bool = True,
                 device: str | torch.device | None = None):
        """
        Initialize the LLMCoder

        Parameters
        ----------
        analyzers : list[str], optional
            The list of analyzers to use, by default []
        model_first : str, optional
            The model to use for the first completion, by default "ft:gpt-3.5-turbo-1106:personal::8LCi9Q0d"
        model_feedback : str, optional
            The model to use for the feedback loop, by default "gpt-3.5-turbo"
        feedback_variant : str, optional
            The feedback variant to use, by default "separate"
        system_prompt : str, optional
            The system prompt to use, by default the one used for preprocessing and fine-tuning
        max_iter : int, optional
            The maximum number of iterations to run the feedback loop, by default 10
        log_conversation : bool, optional
            Whether to log the conversation, by default False
        device : str | torch.device | None, optional
            The device to use for the scoring model, by default the first available GPU or CPU
        """
        # Check for invalid feedback variants
        if feedback_variant not in ["separate", "coworker"]:
            raise ValueError("Inavlid feedback method")
        self.feedback_variant = feedback_variant

        # Remember the models
        self.model_first = model_first
        self.model_feedback = model_feedback

        '''

In [18]:
result = llmcoder.complete(code, temperature=0.9)

Creating first completion...
Considering 14 completions
Choosing message 0 with score 0.2659682631492615
Starting feedback loop...
Starting feedback iteration 1...
Analyzing code in a coworker mode...
Running mypy_analyzer_v1...
No missing stubs found.
/tmp/tmp9jvxnk5_.py:7: error: Skipping analyzing "transformers": module is installed, but missing library stubs or py.typed marker  [import-untyped]
/tmp/tmp9jvxnk5_.py:7: note: See https://mypy.readthedocs.io/en/stable/running_mypy.html#missing-imports
/tmp/tmp9jvxnk5_.py:15: error: Incompatible default for argument "analyzers" (default has type "None", argument has type "list[str]")  [assignment]
/tmp/tmp9jvxnk5_.py:15: note: PEP 484 prohibits implicit Optional. Accordingly, mypy has changed its default to no_implicit_optional=True
/tmp/tmp9jvxnk5_.py:15: note: Use https://github.com/hauntsaninja/no_implicit_optional to automatically upgrade your codebase
Found 2 errors in 1 file (checked 1 source file)

Running signature_analyzer_v1..

In [19]:
print(result)

# Set the device
        if device is None:
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        elif isinstance(device, str):
            self.device = torch.device(device)
        else:
            self.device = device


In [20]:
import re

In [21]:
for message in llmcoder.messages:
    print(message['role'].upper())
    # Remove colors from the content of the message
    message['content'] = re.sub(r'\x1b[^m]*m', '', message['content'])
    print(message['content'])
    print('-' * 80)

SYSTEM
You are AutocompleteGPT, a useful AI autocomplete tool that provides code completions based on the user's code.
You are a precision-focused tool for code autocompletion, adept in languages like Python, JavaScript, C++, and SQL.
Precisely continue the code from the point of interruption and do not repeat or modify the original code, even if it is incorrect or the code interrupts in the middle of a line.
Your code is well documented with comments and annotations, and you should provide a clear explanation of the code's purpose in your code completion.
Your unique capability is to provide completions without altering, repeating, or commenting on the original code.
You offer only the necessary code to complete the snippet, ensuring the response is exclusively code, with no additional comments, explanations, or annotations.
This approach makes you an ideal assistant for users seeking straightforward and efficient code extensions, enhancing their work with accurate, logic-driven compl